In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [ ]:
import random
import pandas as pd
import numpy as np
import os
import glob

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from tqdm.auto import tqdm

import warnings
warnings.filterwarnings(action='ignore') 

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

cpu


In [ ]:
CFG = {
    'EPOCHS':5,
    'LEARNING_RATE':1e-3,
    'BATCH_SIZE':16,
    'SEED':41
}

In [ ]:
all_input_list = sorted(glob.glob('/content/drive/MyDrive/청경채 성장 예측 AI/train_input/*.csv'))
all_target_list = sorted(glob.glob('/content/drive/MyDrive/청경채 성장 예측 AI/train_target/*.csv'))

In [ ]:
train_input_list = all_input_list[:50]
train_target_list = all_target_list[:50]
# print(train_input_list)
# print(train_target_list)

val_input_list = all_input_list[50:]
val_target_list = all_target_list[50:]
# print(val_input_list)
# print(val_target_list)

dataset 생성

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, input_paths, target_paths, infer_mode):
        self.input_paths = input_paths
        self.target_paths = target_paths
        self.infer_mode = infer_mode

        self.data_list = []
        self.label_list = []
        print('Data Pre-processing..')
        for input_path, target_path in tqdm(zip(self.input_paths, self.target_paths)):
            input_df = pd.read_csv(input_path)
            target_df = pd.read_csv(target_path)
            
            input_df = input_df.drop(columns=['시간'])
            input_df = input_df.fillna(0)
            print("input_df : ", input_df)
            
            input_length = int(len(input_df)/1440)
            target_length = int(len(target_df))
            
            for idx in range(target_length):
                time_series = input_df[1440*idx:1440*(idx+1)].values
                self.data_list.append(torch.Tensor(time_series))
            for label in target_df["rate"]:
                self.label_list.append(label)
        print('Done.')
              
    def __getitem__(self, index):
        data = self.data_list[index]
        label = self.label_list[index]
        if self.infer_mode == False:
            return data, label
        else:
            return data
        
    def __len__(self):
        return len(self.data_list)

In [ ]:
train_dataset = CustomDataset(train_input_list, train_target_list, False)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=6)

val_dataset = CustomDataset(val_input_list, val_target_list, False)
val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=6)

Data Pre-processing..


0it [00:00, ?it/s]

input_df :           내부온도관측치    내부습도관측치  CO2관측치     EC관측치    외부온도관측치    외부습도관측치  펌프상태  \
0      24.799999  40.500000   361.0  0.059069  20.299999  33.200001   0.0   
1      20.500000  44.099998   355.0  0.910134  19.700001  10.700000   0.0   
2      20.600000  44.299999   360.0  0.910134  19.799999  10.600000   0.0   
3      20.600000  44.500000   359.0  0.908579  19.799999  10.400000   0.0   
4      20.600000  44.500000   357.0  0.910134  19.799999  10.300000   0.0   
...          ...        ...     ...       ...        ...        ...   ...   
41755  20.400000  78.000000   470.0  0.313223  20.400000  31.500000   0.0   
41756  20.400000  78.199997   471.0  0.314777  20.500000  31.400000   0.0   
41757  20.500000  79.199997   468.0  0.315554  20.600000  31.299999   0.0   
41758  20.600000  79.599998   471.0  0.316332  20.799999  31.200001   0.0   
41759  20.600000  79.900002   476.0  0.316332  20.900000  31.100000   0.0   

       펌프작동남은시간  최근분무량   일간누적분무량  ...  카메라상태  냉방온도  난방온도     기준

0it [00:00, ?it/s]

input_df :           내부온도관측치    내부습도관측치  CO2관측치     EC관측치  외부온도추정관측치  외부습도추정관측치   펌프상태  \
0      22.400000  53.549999   607.0  0.000000  22.799999  29.240000  201.0   
1      22.500000  53.599998   606.0  0.000000  22.799999  29.240001  201.0   
2      22.400000  53.403953   611.0  0.000000  22.799999  29.200001    0.0   
3      22.200001  53.393676   606.0  0.000000  22.680000  29.180001    0.0   
4      22.000000  53.383400   610.0  0.000000  22.600000  29.320000    0.0   
...          ...        ...     ...       ...        ...        ...    ...   
35995  23.400000  84.500000   690.0  4.126665  25.520000  43.180000    0.0   
35996  23.400000  84.400002   692.0  4.127718  25.500000  43.380000    0.0   
35997  23.500000  84.599998   689.0  4.125612  25.480000  43.299998    0.0   
35998  23.500000  84.500000   691.0  4.125612  25.540000  43.419998    0.0   
35999  23.500000  84.400002   691.0  4.124558  25.540000  43.540000    0.0   

       펌프작동남은시간  펌프최근분무량  펌프일간누적분무량  ...  카메라상태  냉방

모델링

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf

모델 정의

In [ ]:
class BaseModel(tf.keras.Model):
  
  def __init__(self):
    self.eval()
    super(BaseModel, self).__init__()
    self.dense1 = tf.keras.layers.Dense(32, input_shape = (37, ), activation=tf.nn.relu)
    self.dense2 = tf.keras.layers.Dense(10, activation=tf.nn.softmax)
    self.dense3 = tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)

  def call(self, inputs):
    x = self.dense1(inputs)
    return self.dense2(x)
    y = self.dense2(inputs)
    return self.dense2(x)

  model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
  tf.keras.Model.eval()


AttributeError: ignored

train

In [ ]:
def train(model, optimizer, train_loader, val_loader, scheduler, device):
    model.to(device)
    criterion = nn.L1Loss().to(device)
    
    best_loss = 9999
    best_model = None
    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []
        for X, Y in tqdm(iter(train_loader)):
            X = X.to(device)
            Y = Y.to(device)
            
            optimizer.zero_grad()
            
            output = model(X)
            loss = criterion(output, Y)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
                    
        val_loss = validation(model, val_loader, criterion, device)
        
        print(f'Train Loss : [{np.mean(train_loss):.5f}] Valid Loss : [{val_loss:.5f}]')
        
        if scheduler is not None:
            scheduler.step()
            
        if best_loss > val_loss:
            best_loss = val_loss
            best_model = model
    return best_model

validation

In [ ]:
def validation(model, val_loader, criterion, device):
    model.eval()
    val_loss = []
    with torch.no_grad():
        for X, Y in tqdm(iter(val_loader)):
            X = X.float().to(device)
            Y = Y.float().to(device)
            
            model_pred = model(X)
            loss = criterion(model_pred, Y)
            
            val_loss.append(loss.item())
            
    return np.mean(val_loss)

In [ ]:
# model

In [ ]:
model = BaseModel()

model

model.eval()
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = None

best_model = train(model, optimizer, train_loader, val_loader, scheduler, device)

AttributeError: ignored

In [ ]:
test_input_list = sorted(glob.glob('/content/drive/MyDrive/청경채 성장 예측 AI/test_input/*.csv'))
test_target_list = sorted(glob.glob('/content/drive/MyDrive/청경채 성장 예측 AI/test_target/*.csv'))

In [ ]:
def inference_per_case(model, test_loader, test_path, device):
    model.to(device)
    model.eval()
    pred_list = []
    with torch.no_grad():
        for X in iter(test_loader):
            X = X.float().to(device)
            
            model_pred = model(X)
            
            model_pred = model_pred.cpu().numpy().reshape(-1).tolist()
            
            pred_list += model_pred
    
    submit_df = pd.read_csv(test_path)
    submit_df['rate'] = pred_list
    submit_df.to_csv(test_path, index=False)

In [ ]:
for test_input_path, test_target_path in zip(test_input_list, test_target_list):
    test_dataset = CustomDataset([test_input_path], [test_target_path], True)
    test_loader = DataLoader(test_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)
    inference_per_case(BaseModel, test_loader, test_target_path, device)

Data Pre-processing..


0it [00:00, ?it/s]

input_df :           내부온도관측치    내부습도관측치  CO2관측치     EC관측치    외부온도관측치    외부습도관측치  펌프상태  \
0      22.100000  19.400000   487.0  0.023535  20.200001  26.799999   0.0   
1      22.100000  19.500000   485.0  0.022630  20.100000  26.799999   0.0   
2      22.100000  19.500000   488.0  0.023535  20.100000  26.700001   0.0   
3      22.100000  19.500000   486.0  0.022630  20.000000  26.799999   0.0   
4      22.100000  19.500000   481.0  0.022630  20.000000  26.900000   0.0   
...          ...        ...     ...       ...        ...        ...   ...   
41755  31.900000  48.599998   523.0  0.000000  32.000000  21.900000   0.0   
41756  31.900000  51.099998   523.0  0.000000  31.700001  22.200001   0.0   
41757  31.799999  52.200001   523.0  0.000000  31.600000  22.299999   0.0   
41758  31.799999  53.200001   535.0  0.000000  31.600000  22.500000   0.0   
41759  31.799999  54.000000   532.0  0.000000  31.600000  22.500000   0.0   

       펌프작동남은시간  최근분무량   일간누적분무량  ...  카메라상태  냉방온도  난방온도  기준온도 

AttributeError: ignored

In [ ]:
model.fit(train_loader, val_loader, epochs=10, batch_size=32)

ValueError: ignored

In [ ]:
model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=0, validation_data(X_val, y_val))

In [ ]:
model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=0, validation_split=0.2))

In [ ]:
model.evaluate(X_test, y_test, batch_size=32)

In [ ]:
model.predict(X_input, batch_size=32)